In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Читаем CSV-файлы:

In [2]:
vm = pd.read_csv('vm.csv', index_col=0)
vmType = pd.read_csv('vmType.csv', index_col=0)

Для простоты не будем рассматривать HDD, так как значения для этого ресурса присутствуют не для всех типов виртуальных машин:

In [3]:
vmType.drop(columns=['hdd'], inplace=True)

Поскольку в нашем случае все машины однородные, то выберем для каждого типа виртуальных машин фиксированные значения используемых ресурсов:

In [4]:
vmType = vmType.groupby('vmTypeId').head(1).copy()

Столбцы id и machineId тоже не нужны:

In [5]:
vmType.drop(columns=['id', 'machineId'], inplace=True)

Сделаем индексацию по типу виртуальной машины:

In [6]:
vmType.set_index('vmTypeId', inplace=True)

Из vm уберём столбцы tenantId и priority, они не понадобятся:

In [7]:
vm.drop(columns=['tenantId', 'priority'], inplace=True)

Добавим к vm значения ресурсов и положим получившуюся таблицу в data:

In [8]:
data = pd.merge(vm, vmType, on="vmTypeId").copy()

Колонка vmTypeId больше не нужна, удалим её:

In [9]:
data.drop(columns=['vmTypeId'], inplace=True)

Копируем data в data2:

In [10]:
data2 = data.copy()

В data удалим endtime, а starttime переименуем в time. В data2 удалим starttime, а endtime переименуем в time.

In [11]:
data.drop(columns=['endtime'], inplace=True)
data.rename(columns={'starttime' : 'time'}, inplace=True)

data2.drop(columns=['starttime'], inplace=True)
data2.rename(columns={'endtime' : 'time'}, inplace=True)

Добавим в каждую из двух таблиц колонку start_or_end. В data проставим значение start, а в data2 проставим значение end.

In [12]:
data['start_or_end'] = 'start'

data2['start_or_end'] = 'end'

Соединим две таблицы:

In [13]:
data = pd.concat([data, data2], ignore_index=True).copy()

Все строки с NaN (они есть только в столбце time) уберём, так как это просто значит, что виртуальная машина продолжает работать:

In [14]:
data.dropna(inplace=True)

Отсортируем записи по времени, чтобы построчным чтением можно было имитировать динамические запросы:

In [15]:
data.sort_values(by=['time'], inplace=True, ignore_index=True)

Переставим колонки для приведения к требуемому формату:

In [16]:
data = data[['vmId', 'start_or_end', 'time', 'core', 'memory', 'ssd', 'nic']].copy()

Сгенерируем итоговые CSV-файлы:

In [17]:
import random

In [18]:
random.seed(1011)
for i in range(100):
    start_slice = random.randint(0, 10549316 - 10000)
    end_slice = start_slice + 10000 - 1
    data_slice = data.loc[start_slice:end_slice].copy()
    data_slice.to_csv("slices/slice" + str(i + 1) + ".csv", header=False, index=False)